<a href="https://colab.research.google.com/github/Shota-8811396/output/blob/output/%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E5%AE%9F%E8%A3%85%E2%91%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### データセットの準備

#### 1.データセットの読み込み

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import sklearn
sklearn.__version__

'1.2.2'

In [3]:
from sklearn.datasets import load_breast_cancer

In [4]:
# データセットの読み込み
breast_cancer = load_breast_cancer()

In [5]:
# 入力値と目標値を抽出
x = breast_cancer['data']
t = breast_cancer['target']

In [6]:
x.shape

(569, 30)

In [7]:
t.shape

(569,)

In [8]:
type(x), type(t)

(numpy.ndarray, numpy.ndarray)

In [9]:
# ndarry -> Tensor へ変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [10]:
type(x), type(t)

(torch.Tensor, torch.Tensor)

In [11]:
x.dtype, t.dtype

(torch.float32, torch.int64)

In [12]:
x.shape, t.shape

(torch.Size([569, 30]), torch.Size([569]))

- 入力値： float32
- 目標値
  - 回帰： float32
  - 二値分類： float32
  - 多値分類： int64

#### 2.DataLoader
Data Loader は PyTorch でモデルの訓練を行う際に利用することが多い。

ニューラルネットワークではミニバッチ学習を行うため、データセットのサンプルをランダムに並べ替えて、バッチサイズごとにまとめる。

ここを担当するのが Data Loader 。

Data Loader がこの処理を引き受けるため、モデルの訓練を行うときにミニバッチ学習の細々とした処理を意識することなく進めることができる。

Data Loader を利用する前に、入力値 x と目標値 t を合わせた TensorDataset に変換する必要があり、それを dataset として格納する。

In [16]:
# 入力値と目標値を dataset としてまとめる
dataset = torch.utils.data.TensorDataset(x, t)
dataset

In [17]:
type(dataset)

torch.utils.data.dataset.TensorDataset

In [18]:
dataset[0]

(tensor([1.7990e+01, 1.0380e+01, 1.2280e+02, 1.0010e+03, 1.1840e-01, 2.7760e-01,
         3.0010e-01, 1.4710e-01, 2.4190e-01, 7.8710e-02, 1.0950e+00, 9.0530e-01,
         8.5890e+00, 1.5340e+02, 6.3990e-03, 4.9040e-02, 5.3730e-02, 1.5870e-02,
         3.0030e-02, 6.1930e-03, 2.5380e+01, 1.7330e+01, 1.8460e+02, 2.0190e+03,
         1.6220e-01, 6.6560e-01, 7.1190e-01, 2.6540e-01, 4.6010e-01, 1.1890e-01]),
 tensor(0))

In [19]:
dataset[0][0]

tensor([1.7990e+01, 1.0380e+01, 1.2280e+02, 1.0010e+03, 1.1840e-01, 2.7760e-01,
        3.0010e-01, 1.4710e-01, 2.4190e-01, 7.8710e-02, 1.0950e+00, 9.0530e-01,
        8.5890e+00, 1.5340e+02, 6.3990e-03, 4.9040e-02, 5.3730e-02, 1.5870e-02,
        3.0030e-02, 6.1930e-03, 2.5380e+01, 1.7330e+01, 1.8460e+02, 2.0190e+03,
        1.6220e-01, 6.6560e-01, 7.1190e-01, 2.6540e-01, 4.6010e-01, 1.1890e-01])

In [20]:
dataset[0][1]

tensor(0)

- 訓練データ： ネットワークのパタメータの最適化
- 検証データ: ネットワークが持つハイパーパラメータの最適化
- テストデータ: 訓練済みネットワークの評価

In [22]:
# 各データセットのサンプル数を決定
# trainl : val : test = 60% : 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

n_train, n_val, n_test

(341, 113, 115)

In [23]:
# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

In [25]:
# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [26]:
# サンプル数を確認
len(train), len(val), len(test)

(341, 113, 115)

In [27]:
# バッチサイズ
batch_size = 10

In [30]:
# DataLoader の定義
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

In [32]:
x, t = next(iter(train_loader))

In [33]:
x.shape

torch.Size([10, 30])

In [34]:
t.shape

torch.Size([10])